<a href="https://colab.research.google.com/github/RaianaSeixas/NLT/blob/main/Relevant_Job__Title_Pictures_v11_based_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Determining Relevant Job Title Pictures**
The objective of this short project is to automatically find relevant pictures for job titles. We are looking for pictures that represent the activities typically associated with that job title, showing people or tools used on the job.
## Dataset
The dataset provided is a .csv file with a set of pictures queried from Google Images, with the following available metadata and pre-extracted features:

* Job Title: the job we are trying to get pictures for.

* Pic Title: the picture title according to Google Images.

* Num Resumes: the number of resumes we have available for the job title. This indicates how popular /common the job title is.

* Google Position: the position of the image in the search results.

* Strict Face Count: face count according to a high-precision face recognition algorithm from OpenCV. When it detects a face, it is a human face almost 100% of the time, but it doesn't detect all human faces.

* Relaxed Face Count: face count according to a high-recall face recognition algorithm from OpenCV. It detects human faces almost 100% of the time, but it also confuses some other shapes and objects with human faces.

* KB size: size of the image in kilobytes.

* Height: pixel height of the image.

* Width: pixel width of the image.

* Resolution: total pixels in the image.

* Text regions: number of text regions identified in the image, also using OpenCV.

* Picture URL: the URL of the image.

* Manual Label: a label indicating if the picture is good (relevant) or bad (irrelevant). 

##Methodology
The feature "Manual Label" is available only for  around 20% of the dataset. For this reason, I decided to use unsupervised learning to create the model and use the labaled data to evaluate the quality of the model.
----------
Author: Raiana Roland Seixas
09/07/2022

# Data load & Libraries

In [255]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [256]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.cluster import KMeans
from sklearn.neural_network import BernoulliRBM

In [257]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score

In [258]:
!git clone https://github.com/RaianaSeixas/NLT
%cd NLT

Cloning into 'NLT'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.
/content/NLT/NLT/NLT/NLT/NLT/NLT/NLT/NLT/NLT


In [259]:
df=pd.read_csv('occupation_pictures.csv')
df

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Picture URL,Manual Label
0,3D Modeler,Our Top 19 3D Modeling Software Picks ...,1261,0,0,7,276.763672,1917,2301,4411017,2,https://format-magazine-production-res.cloudin...,NaN
1,3D Modeler,7 Tips for Beginner 3D Modelers ...,1261,1,0,3,44.343750,340,747,253980,5,http://static1.squarespace.com/static/5320f582...,NaN
2,3D Modeler,How to Become an Expert 3D Modeler ...,1261,2,0,2,43.088867,380,660,250800,8,https://www.gamedesigning.org/wp-content/uploa...,NaN
3,3D Modeler,What is 3D Modeling & What's It Used For?,1261,3,0,2,78.623047,330,750,247500,1,https://cdn.conceptartempire.com/images/04/522...,NaN
4,3D Modeler,Shoofping 3D Modeler ...,1261,4,0,1,97.517578,1075,1909,2052175,3,http://shoofping.com/wp-content/uploads/2017/1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,Wind Turbine Engineer,Wind turbine technician fastest-growing ...,61,4,0,0,89.249023,720,1280,921600,0,https://i.ytimg.com/vi/lUf2mOYp1d4/maxresdefau...,good
654,Wind Turbine Engineer,How to Become a Wind Energy Engineer ...,61,5,0,0,37.949219,200,300,60000,0,https://www.environmentalscience.org/wp-conten...,NaN
655,Wind Turbine Engineer,Wind - Get Into Energy,61,6,0,1,324.730469,500,750,375000,1,http://getintoenergy.com/wp-content/uploads/20...,good
656,Wind Turbine Engineer,Wind-energy programs yield engineering ...,61,7,0,1,36.197266,920,613,563960,0,https://s.hdnux.com/photos/20/15/22/4248747/3/...,good


In [260]:
documents = list(df['Pic Title'].values)
job = list(df['Job Title'].values)

In [261]:
documents

['Our Top 19 3D Modeling Software Picks ...',
 '7 Tips for Beginner 3D Modelers ...',
 'How to Become an Expert 3D Modeler ...',
 "What is 3D Modeling & What's It Used For?",
 'Shoofping 3D Modeler ...',
 'How to Become a Professional 3D Modeler ...',
 'Top 13 of the best online 3D modeling ...',
 'METHODS TO ELEVATE AS A 3D MODELER ...',
 'Admissions Coordinator Resume Samples ...',
 'Top 10 admissions coordinator interview ...',
 'Admissions Coordinator Resume Samples ...',
 'Admissions Coordinator Resume Samples ...',
 'Admissions Coordinator Resume Samples ...',
 'Admissions Coordinator Resume Samples ...',
 'Marine Air Support Squadron 2 Marines ...',
 'Air Support Operations Course',
 'CHIPS Articles: At Combined Resolve ...',
 '2nd LAAD, 22 MEU Stinger missle ...',
 '8th ASOS trains with Slovenian TACPs ...',
 'Marines, Corpsmen Reassess Needs',
 'Architectural Drafter Career ...',
 'Here are 9 firms offering architecture ...',
 'architectural drafter by Michael Reisman',
 'What

In [262]:
def text_variable(documents,cluster_k):
  vectorizer = TfidfVectorizer(stop_words='english')
  XW = vectorizer.fit_transform(documents)

  true_k = 8
  model = KMeans(n_clusters=cluster_k, init='k-means++', max_iter=100, n_init=1)
  model.fit(XW)
  resp = []
  for x in documents:
    doc=[]
    doc.append(x)
    Y = vectorizer.transform(doc)
    prediction = model.predict(Y)
    resp.append(prediction[0])
  return resp

In [263]:
def text_variable_Birch(documents,cluster_k):
  vectorizer = TfidfVectorizer(stop_words='english')
  XW = vectorizer.fit_transform(documents)

  model = Birch(threshold=0.01, n_clusters = cluster_k)
  model.fit(XW)
  resp = []
  for x in documents:
    doc=[]
    doc.append(x)
    Y = vectorizer.transform(doc)
    prediction = model.predict(Y)
    resp.append(prediction[0])
  return resp

In [264]:
data=df.drop(columns=['Picture URL'])

In [265]:
data['Manual Label'].describe()

count      133
unique       2
top       good
freq        91
Name: Manual Label, dtype: object

In [266]:
data['Pic Title'].describe()

count                            658
unique                           593
top       Bureau of Labor Statistics
freq                              17
Name: Pic Title, dtype: object

# Exploratory Analysis

# Data Preprocessing

In [267]:
import nltk
ps = nltk.PorterStemmer()
wn=nltk.WordNetLemmatizer()
nltk.download("popular")
stopwords = nltk.corpus.stopwords.words('english')
import re
import string

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

### Clean up text

In [268]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

data['Job Title'] = data['Job Title'].apply(lambda x: clean_text(x.lower()))
data['Pic Title'] = data['Pic Title'].apply(lambda x: clean_text(x.lower()))
data.head()

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label
0,"[3d, modeler]","[top, 19, 3d, modeling, software, picks, ]",1261,0,0,7,276.763672,1917,2301,4411017,2,NaN
1,"[3d, modeler]","[7, tips, beginner, 3d, modelers, ]",1261,1,0,3,44.343750,340,747,253980,5,NaN
2,"[3d, modeler]","[become, expert, 3d, modeler, ]",1261,2,0,2,43.088867,380,660,250800,8,NaN
3,"[3d, modeler]","[3d, modeling, whats, used]",1261,3,0,2,78.623047,330,750,247500,1,NaN
4,"[3d, modeler]","[shoofping, 3d, modeler, ]",1261,4,0,1,97.517578,1075,1909,2052175,3,NaN


In [269]:
data.columns

Index(['Job Title', 'Pic Title', 'Num Resumes', 'Google Position',
       'Strict Face Count', 'Relaxed Face Count', 'KB size', 'Height', 'Width',
       'Resolution', 'Text Regions', 'Manual Label'],
      dtype='object')

### Stem / Lemmatize text

In [270]:
def stemming(text):
  stemmed=[ps.stem(word) for word in text]
  return stemmed

data['Stem Job Title'] = data['Job Title'].apply(lambda x: stemming(x))
data['Stem Pic Title'] = data['Pic Title'].apply(lambda x: stemming(x))
data

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
0,"[3d, modeler]","[top, 19, 3d, modeling, software, picks, ]",1261,0,0,7,276.763672,1917,2301,4411017,2,NaN,"[3d, model]","[top, 19, 3d, model, softwar, pick, ]"
1,"[3d, modeler]","[7, tips, beginner, 3d, modelers, ]",1261,1,0,3,44.343750,340,747,253980,5,NaN,"[3d, model]","[7, tip, beginn, 3d, model, ]"
2,"[3d, modeler]","[become, expert, 3d, modeler, ]",1261,2,0,2,43.088867,380,660,250800,8,NaN,"[3d, model]","[becom, expert, 3d, model, ]"
3,"[3d, modeler]","[3d, modeling, whats, used]",1261,3,0,2,78.623047,330,750,247500,1,NaN,"[3d, model]","[3d, model, what, use]"
4,"[3d, modeler]","[shoofping, 3d, modeler, ]",1261,4,0,1,97.517578,1075,1909,2052175,3,NaN,"[3d, model]","[shoofp, 3d, model, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,"[wind, turbine, engineer]","[wind, turbine, technician, fastestgrowing, ]",61,4,0,0,89.249023,720,1280,921600,0,good,"[wind, turbin, engin]","[wind, turbin, technician, fastestgrow, ]"
654,"[wind, turbine, engineer]","[become, wind, energy, engineer, ]",61,5,0,0,37.949219,200,300,60000,0,NaN,"[wind, turbin, engin]","[becom, wind, energi, engin, ]"
655,"[wind, turbine, engineer]","[wind, get, energy]",61,6,0,1,324.730469,500,750,375000,1,good,"[wind, turbin, engin]","[wind, get, energi]"
656,"[wind, turbine, engineer]","[windenergy, programs, yield, engineering, ]",61,7,0,1,36.197266,920,613,563960,0,good,"[wind, turbin, engin]","[windenergi, program, yield, engin, ]"


In [271]:
#data[data['Manual Label']=='bad']

In [272]:
#data[data['Manual Label']=='good']

In [273]:
documents = list(data['Stem Pic Title'].values)
job = list(data['Stem Job Title'].values)

In [274]:
documents

[['top', '19', '3d', 'model', 'softwar', 'pick', ''],
 ['7', 'tip', 'beginn', '3d', 'model', ''],
 ['becom', 'expert', '3d', 'model', ''],
 ['3d', 'model', 'what', 'use'],
 ['shoofp', '3d', 'model', ''],
 ['becom', 'profession', '3d', 'model', ''],
 ['top', '13', 'best', 'onlin', '3d', 'model', ''],
 ['method', 'elev', '3d', 'model', ''],
 ['admiss', 'coordin', 'resum', 'sampl', ''],
 ['top', '10', 'admiss', 'coordin', 'interview', ''],
 ['admiss', 'coordin', 'resum', 'sampl', ''],
 ['admiss', 'coordin', 'resum', 'sampl', ''],
 ['admiss', 'coordin', 'resum', 'sampl', ''],
 ['admiss', 'coordin', 'resum', 'sampl', ''],
 ['marin', 'air', 'support', 'squadron', '2', 'marin', ''],
 ['air', 'support', 'oper', 'cours'],
 ['chip', 'articl', 'combin', 'resolv', ''],
 ['2nd', 'laad', '22', 'meu', 'stinger', 'missl', ''],
 ['8th', 'aso', 'train', 'slovenian', 'tacp', ''],
 ['marin', 'corpsmen', 'reassess', 'need'],
 ['architectur', 'drafter', 'career', ''],
 ['9', 'firm', 'offer', 'architectur', 

In [275]:
arr1 = data['Stem Job Title'].values # first information (common small)
arr2 = data['Stem Pic Title'].values # second information
n = arr1.shape[0] # n = 658
lista = []
divide = []
for i in range(n):
  aux =0
  c1 = list(arr1[i])
  for x in arr1[i]:
    c2 = list(arr2[i])
    if(c2.count(x) != 0):
      aux = aux + 1
  lista.append(aux)
  try:
   divide.append(len(c1)/aux)
  except:
   divide.append(0)
print(lista)
print(divide)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 0, 0, 0, 0, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 3, 3, 2, 3, 3, 3, 2, 0, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 1, 2, 2, 2, 2, 4, 4, 2, 2, 4, 1, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 1, 0, 2, 2, 1, 0, 3, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 2, 0, 2, 2, 2, 2, 0, 0, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 0, 3, 3, 0, 2, 4, 0, 1, 0, 2, 0, 0, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 3, 0, 2, 0, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 1, 1, 0, 0, 1, 1, 1, 2, 3, 1, 3, 3, 3, 2, 3, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 

In [276]:
arr1 = 'career'
arr2 = data['Stem Pic Title'].values # second information
n = arr2.shape[0] # n = 658
career = []
for i in range(n):
  aux =0
  c1 = arr1
  c2 = list(arr2[i])
  if(c2.count(c1) != 0):
    career.append(1)
  else:
    career.append(0)
print(career)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [277]:
'''
arr1 = data['Stem Job Title'].values # first information (common small)
arr2 = data['Stem Pic Title'].values # second information
n = arr1.shape[0] # n = 658
lista = []
divide = []
verifica = []
for i in range(n):
  aux =0
  c1 = list(arr1[i])
  for x in arr1[i]:
    c2 = list(arr2[i])
    t1 = x[:4]
    for t2 in c2:
      if(t2[:4].count(t1) != 0):
        aux = aux + 1
  lista.append(aux)
  try:
   divide.append(len(c1)/aux)
  except:
   divide.append(0)
  verifica.append([t2[:4],t1])
print(lista)
print(divide)
'''

"\narr1 = data['Stem Job Title'].values # first information (common small)\narr2 = data['Stem Pic Title'].values # second information\nn = arr1.shape[0] # n = 658\nlista = []\ndivide = []\nverifica = []\nfor i in range(n):\n  aux =0\n  c1 = list(arr1[i])\n  for x in arr1[i]:\n    c2 = list(arr2[i])\n    t1 = x[:4]\n    for t2 in c2:\n      if(t2[:4].count(t1) != 0):\n        aux = aux + 1\n  lista.append(aux)\n  try:\n   divide.append(len(c1)/aux)\n  except:\n   divide.append(0)\n  verifica.append([t2[:4],t1])\nprint(lista)\nprint(divide)\n"

In [278]:
# print(verifica)

In [279]:
for x in c2:
  print(x[:4])

care
map
wind
tech



In [280]:
data.head()

,Job Title,Pic Title,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
0,"[3d, modeler]","[top, 19, 3d, modeling, software, picks, ]",1261,0,0,7,276.763672,1917,2301,4411017,2,NaN,"[3d, model]","[top, 19, 3d, model, softwar, pick, ]"
1,"[3d, modeler]","[7, tips, beginner, 3d, modelers, ]",1261,1,0,3,44.343750,340,747,253980,5,NaN,"[3d, model]","[7, tip, beginn, 3d, model, ]"
2,"[3d, modeler]","[become, expert, 3d, modeler, ]",1261,2,0,2,43.088867,380,660,250800,8,NaN,"[3d, model]","[becom, expert, 3d, model, ]"
3,"[3d, modeler]","[3d, modeling, whats, used]",1261,3,0,2,78.623047,330,750,247500,1,NaN,"[3d, model]","[3d, model, what, use]"
4,"[3d, modeler]","[shoofping, 3d, modeler, ]",1261,4,0,1,97.517578,1075,1909,2052175,3,NaN,"[3d, model]","[shoofp, 3d, model, ]"


career, senior

In [281]:
'''def lemmatized(text):
  lemmatized= [wn.lemmatize(word) for word in text]
  return lemmatized

data['Lemma Job Title'] = data['Job Title'].apply(lambda x: lemmatized(x))
data['Lemma Pic Title'] = data['Pic Title'].apply(lambda x: lemmatized(x))
data'''

"def lemmatized(text):\n  lemmatized= [wn.lemmatize(word) for word in text]\n  return lemmatized\n\ndata['Lemma Job Title'] = data['Job Title'].apply(lambda x: lemmatized(x))\ndata['Lemma Pic Title'] = data['Pic Title'].apply(lambda x: lemmatized(x))\ndata"

###Data split

In [282]:
data=data.drop(columns=['Job Title','Pic Title'])

In [283]:
''' Data (with labels) to test the model '''

labaled= data[data['Manual Label'].notna()]
labaled.head()

,Num Resumes,Google Position,Strict Face Count,Relaxed Face Count,KB size,Height,Width,Resolution,Text Regions,Manual Label,Stem Job Title,Stem Pic Title
21,1824,1,0,2,529.085938,1382,2200,3040400,2,good,"[architectur, drafter]","[9, firm, offer, architectur, ]"
24,1824,4,0,1,28.923828,446,600,267600,0,good,"[architectur, drafter]","[2017, architectur, drafter, careercastcom]"
29,347272,2,0,0,56.383789,640,1280,819200,1,bad,[assist],"[googl, assist, 20, chang, ]"
32,347272,7,1,2,390.638672,844,1243,1049092,0,good,[assist],"[person, assist, app, ]"
39,868,0,1,10,640.045898,1500,2000,3000000,0,good,"[assist, curat]","[assist, curat, ancient, ]"


In [284]:
#Drop target feature
#X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title','KB size','Num Resumes','Google Position','Width','Text Regions']) 
X = data.drop(columns=['Height','Manual Label','Stem Job Title','Stem Pic Title','KB size','Resolution','Width']) 
#'Num Resumes','Google Position','Text Regions'
#X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title']) 
#X['Common']=lista
#X['Division']=divide
#X['Career'] = career
ncluster = 8
resp = text_variable(documents,ncluster)
X['Doc_title'] = resp
ncluster = 5
resp = text_variable(job,ncluster)
X['Doc_job'] = resp
W = X
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

AttributeError: ignored

# Predictive Model

In [ ]:
#y_pred = KMeans(n_clusters=2).fit_predict(X)
y_pred = KMeans(n_clusters=2, init='k-means++', max_iter=200, n_init=1).fit_predict(X)

In [ ]:
len(y_pred)

In [ ]:
# get the elements containing label (we will use it to evaluate the method)
y = data['Manual Label'].replace({'good':0, 'bad':1})
index_no = data['Manual Label'].notna()
y_real = y[index_no].astype(int)
X_eval= X[index_no,:]
y_mod = y_pred[index_no]

In [ ]:
y_real.shape, y_mod.shape

In [ ]:
# Classification Report
print('\n\n', classification_report(y_real, y_mod, target_names=['0-Good', '1-Bad']))
# Confusion Matrix
con_matrix = pd.DataFrame(confusion_matrix(y_real, y_mod), columns=['Predicted Good', 'Predicted Bad'], index=['Actual Good', 'Actual Bad'])
  
plt.figure(figsize=(15,10))
sns.heatmap(data=con_matrix, cmap='cool');
plt.title('Model Confusion Matrix')
plt.show();

In [ ]:
# Birch:  Balanced Iterative Reducing and Clustering using Hierarchies
# “threshold” and “n_clusters” are the hyperparameters

In [ ]:
from sklearn.cluster import Birch


In [ ]:
#Drop target feature
X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title','KB size','Resolution','Width']) 
#'Google Position','Text Regions','Num Resumes','Height',
#X = data.drop(columns=['Manual Label','Stem Job Title','Stem Pic Title']) 
#X['Common']=lista
#X['Division']=divide
#X['Career'] = career
ncluster = 5
# resp = text_variable(documents,ncluster)
resp = text_variable_Birch(documents,ncluster)
X['Doc_title'] = resp
ncluster = 8
#resp = text_variable(job,ncluster)
resp = text_variable_Birch(job,ncluster)
X['Doc_job'] = resp
W = X
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
model = Birch(threshold=0.01, n_clusters=2)
# fit the model
model.fit(X)
# assign a cluster to each example
y_calc = model.predict(X)
index_no = data['Manual Label'].notna()
y_mod = y_calc[index_no]

In [ ]:

# Classification Report
print('\n\n', classification_report(y_real, y_mod, target_names=['0-Good', '1-Bad']))
# Confusion Matrix
con_matrix = pd.DataFrame(confusion_matrix(y_real, y_mod), columns=['Predicted Good', 'Predicted Bad'], index=['Actual Good', 'Actual Bad'])
  
plt.figure(figsize=(15,10))
sns.heatmap(data=con_matrix, cmap='cool');
plt.title('Model Confusion Matrix')
plt.show();

In [ ]:
W

In [ ]:
#y = data['Manual Label'].replace({'good':0, 'bad':1})
lista = []
for yi in y_calc:
  if(yi == 0):
    lista.append('good')
  else:
    lista.append('bad')

In [ ]:
df['Predict Label'] = lista

In [ ]:
Arq = "output_excel.xlsx"
df.to_excel(Arq)
from google.colab import files
files.download(Arq)

In [ ]:
Arq = "output_csv.csv"
df.to_csv(Arq)
from google.colab import files
files.download(Arq)